# HDP Gibbs Samplers

## *( 5.1 )* Posterior Sampling in the Chinese Restaurant Franchise

### Brief Overview

The Hierarchical Dirichlet Process mixture model is given by
$$\begin{aligned}
G_0 | \gamma, H &\sim DP(\gamma, H) \\
G_j | \alpha_0, G_0 &\sim DP(\alpha_0, G_0) \\
\theta_{ji} | G_j &\sim G_j \\
x_{ji} | \theta_{ji} &\sim F(\theta_{ji})
\end{aligned} $$

This model is able to non-parametrically cluster each group's data while sharing information both between and within groups.  A Dirichlet process is essentially a discrete distribution with atoms drawn from a (not-necessarily discrete) base measure $H$ and gradually decreasing weights determined by the "stick-breaking process."  In the HDP, each group is a Dirichlet process drawn from another DP $G_0$, so these will contain the same atoms as $G_0$ but with different weights:
$$\begin{aligned}
G_0 &= \sum_{k=1}^{\infty} \beta_k \delta(\phi_k) \\
G_j &= \sum_{k=1}^{\infty} \pi_{jk} \delta(\phi_k) \\
\phi_k | H &\sim H
\end{aligned} $$
Additionally, if we define $\beta, \pi_j$ as the collected weights above, it can be shown that these vectors encode a distribution over $\mathbb{Z}^+$ such that $\beta | \gamma \sim GEM(\gamma)$ and $\pi_j | \alpha_0, \beta \sim DP(\alpha_0, \beta)$.

Successive draws from a DP exhibit clustering behavior, since the probability of taking a certain value is a related to the number of previous draws of that value.  This is shown in the hierarchical sense by the *Chinese restaurant franchise* process.  Imagine a group of Chinese restaurants with a certain number of tables at each restaurant.  Let $\phi_k$ be the global dishes, drawn from $H$; $\psi_{jt}$ be the table-specific dishes, drawn from $G_0$; and $\theta_{ji}$ be the customer-specific dishes, drawn from $G_j$.  Denote $z_{ji}$ as the dish index eaten by customer $ji$; $t_{ji}$ as the table index where customer $ji$ sits; $k_{jt}$ be the dish index served at table $jt$; $n_{jtk}$ be the customer counts; and $m_{jk}$ be the table counts.  Then:

$$\begin{aligned}
\theta_{ji} | \text{other } \theta, \alpha_0, G_0 &\sim
    \sum_{t=1}^{m_{j\cdot}} \frac{n_{jt\cdot}}{i-1+\alpha_0} \delta(\psi_{jt}) +
                            \frac{\alpha_0}{i-1+\alpha_0} G_0 \\
\psi_{jt} | \text{other } \psi, \gamma, H &\sim
    \sum_{k=1}^{K} \frac{m_{\cdot k}}{m_{\cdot \cdot} + \gamma} \delta(\phi_k) +
                            \frac{\gamma}{m_{\cdot \cdot} + \gamma} H
\end{aligned} $$

### Full Conditionals

Choose some base measure $h(\cdot)$ and a conjugate data-generating distribution $f(\cdot | \theta)$.  Important to compute are $f_k^{-x_{ji}}(x_{ji})$, the mixture component of customer $ij$ under $k$, and $f_k^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt})$, the mixture component of table $jt$ under $k$.  This is done by integrating out $\phi_k$ over the joint density of all such points, for example:

$$\begin{aligned}
f_k^{-x_{ji}}(x_{ji}) &= \frac { \int f(x_{ij} | \phi_k) g(k)d\phi_k } { \int g(k)d\phi_k } \\
g(k) &= h(\phi_k) \prod_{j'i' \neq ji, z_{j'i'} = k} f(x_{j'i'} | \phi_k) 
\end{aligned} $$

The corresponding mixture components for a new customer assignment and new table assignment are denoted $f_{k^*}^{-x_{ji}}(x_{ji})$ and $f_{k^*}^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt})$, which are special cases of their the respective $f_k$ component where no data points have $z_{ij} = k^*$.

Using this, we first compute the likelihood of a given point $x_{ji}$ given the current clustering scheme:
$$
p(x_{ji} | t^{-ji}, t_{ji} = t^*, k) =
    \sum_{k=1}^{K} \frac{m_{\cdot k}}{m_{\cdot \cdot} + \gamma} f_k^{-x_{ji}}(x_{ji}) +
                            \frac{\gamma}{m_{\cdot \cdot} + \gamma} f_{k^*}^{-x_{ji}}(x_{ji})
$$

For efficiency, the Gibbs scheme implemented below only samples the $t$ and $k$ indexes (which can later be reverse-engineered to obtain the actual parameters).  The state space of the $k$ values is technically infinite, and the number of tables/dishes currently associated with the data is undefined.  We keep a running list of active $t$ and $k$ values.  Each update step, each customer is assigned either to one of the existing tables or to a new table, and if a customer is assigned to a new table, a new $k$ corresponding value gets drawn; similarly, each table is assigned a dish, either from the existing dishes or with a new dish.  If a table/dish becomes unrepresented in the current scheme, it gets removed from its respective list.  The update full conditionals are:

$$ \begin{aligned}
p(t_{ji} = t | t^{-ji}, k, ...) &\propto \begin{cases}
    n_{jt\cdot}^{-ji} f_{k_{jt}}^{-x_{ji}}(x_{ji}) & t\text{ used}\\
    \alpha_0 p(x_{ji} | ...) & t\text{ new}
    \end{cases} \\
p(k_{jt} = k | t, k^{-jt}) &\propto \begin{cases}
    m_{\cdot k} f_k^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt}) & k\text{ used}\\
    \gamma f_{k^*}^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt}) & k\text{ new}
    \end{cases} \\
\end{aligned} $$

### Distribution-Specific Mixture Components

The only part of this sampling algorithm that depends on the choice of the measures $H$ and $F$ are the mixture components $f_k$, so this is the only part that needs rewritten for each type of model.  Let
$$ \begin{aligned}
V_{kji} &= \{ j'i' : j'i' \neq ji, z_{j'i'} = k \} \\
W_{kjt} &= \{ j'i' : j't_{j'i'} \neq jt, k_{j't_{j'i'}} = k \} \\
T_{jt} &= \{ j'i': t_{j'i'} = jt \} \\
\end{aligned} $$
$V$ is the set of all customers (excluding customer $ij$) eating dish $k$; $W$ is the set of all customers at tables (excluding table $jt$) eating $k$; these correspond to the product terms in the mixture components.  By conjugacy rules and kernel tricks, each $f_k$ can be expressed as functions of these sets.  Each $f_{k^*}$ can be found by using the corresponding $f_k$ formula where $V$ or $W$ is the empty set.

***F = Poisson, H = Gamma***

$$ \begin{aligned}
f(x | \phi_k) &\sim Poisson(\phi_k) \\
h(\phi_k) &\sim Gamma(\alpha, \beta) \\
\\
f_k^{-x_{ji}}(x_{ji}) &= \frac{1}{x_{ji}!} \cdot
    \frac{\Gamma(x_{ji} + \alpha_v)}{(1 + \beta_v)^{x_{ji} + \alpha_v}} \cdot
    \frac{(\beta_v)^{\alpha_v}}{\Gamma(\alpha_v)} \\
f_k^{-\mathbf{x}_{jt}}(\mathbf{x}_{jt}) &= \frac{1}{\prod_T x_t!} \cdot
    \frac{\Gamma(\sum_T x_t + \alpha_w)}{(|T| + \beta_w)^{\sum_T x_t + \alpha_w}} \cdot
    \frac{(\beta_w)^{\alpha_w}}{\Gamma(\alpha_w)} \\
\alpha_v &= \sum_V x_v + \alpha \quad , \quad \beta_v = |V| + \beta \\
\alpha_w &= \sum_W x_w + \alpha \quad , \quad \beta_w = |W| + \beta \\
\end{aligned} $$

***F = Multinomial, H = Dirichlet***

Let $\mathbf{x}$ be a feature vector of length $L$.  The Multinomial/Dirichlet model is given by
$$ \begin{aligned}
f(\mathbf{x} | n, \mathbf{\phi}_k) &\sim Multinomial(n, \mathbf{\phi}_k) \\
h(\mathbf{\phi}_k) &\sim Dirichlet(L, \mathbf{\alpha}) \\
\end{aligned} $$
Note that each $\mathbf{x}_{ji}$ can have a different value of $n_{ji}$, representing different size draws from the multinomial distribution.  But $\phi_k$, representing the relative concentrations of the $L$ components, will be the same.

$$ \begin{aligned}
f_k^{-\mathbf{x}_{ji}}(\mathbf{x}_{ji}) &= \frac{n_{ji}!}{\prod_{\ell=1}^L (\mathbf{x}_{ji})_\ell!} \cdot
    \frac{ \prod \Gamma(\mathbf{\alpha}_{\ell}^{top}) }{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{top}) } \cdot
    \frac{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{bottom}) }{ \prod \Gamma(\mathbf{\alpha}_{\ell}^{bottom}) } \\
\mathbf{\alpha}_{\ell}^{bottom} &= \sum_V (\mathbf{x}_v)_{\ell} + \mathbf{\alpha}_{\ell} \\
\mathbf{\alpha}_{\ell}^{top} &= (\mathbf{x}_{ji})_{\ell} + \mathbf{\alpha}_{\ell}^{bottom} \\
\\
f_k^{-\mathbf{X}_{jt}}(\mathbf{X}_{jt}) &=
    \frac{ \prod_T n_t! }{ \prod_T \prod_{\ell=1}^L (\mathbf{x}_t)_\ell! } \cdot
    \frac{ \prod \Gamma(\mathbf{\alpha}_{\ell}^{top}) }{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{top}) } \cdot
    \frac{ \Gamma(\sum \mathbf{\alpha}_{\ell}^{bottom}) }{ \prod \Gamma(\mathbf{\alpha}_{\ell}^{bottom}) } \\
\mathbf{\alpha}_{\ell}^{bottom} &= \sum_W (\mathbf{x}_w)_{\ell} + \mathbf{\alpha}_{\ell} \\
\mathbf{\alpha}_{\ell}^{top} &= \sum_T (\mathbf{x}_t)_\ell + \mathbf{\alpha}_{\ell}^{bottom} \\
\end{aligned} $$

In the application of latent topic modeling for NLP, restaurant $j$ represents document $j$, and customer $ji$ represents some subset of the word counts in document $j$, where $L$ is the size of the entire vocabulary for the corpus of documents.  If a customer is one word, $\mathbf{x}_{ji}$ is a draw from a categorical distribution of size $L$, a special case of the multinomial; but a customer can also represent the set of all instances of a unique word, the set of words in a sentence, the set of words in a paragraph, etc.

The dishes $k$ represent topics (where $\mathbf{\phi}_k$ is a distribution over the vocabulary), and the tables $t$ represent clusters of words, sentences, paragraphs (however a "customer" is encoded) from a single document.  Due to the nature of the HDP, these topics are shared among documents and among clusters within a document.  If the customer subsets are mutually exclusive, then each section of a document will be assigned to exactly one topic; but if the customers overlap, the same section of a document could be assigned to multiple topics.

### Sampler Implementation

In [200]:
import numpy as np
import pandas as pd
from scipy.special import gammaln as logg
from sklearn.preprocessing import normalize

In [201]:
def pois_fk_cust(i, x, k, Kmax, ha, hb, new=False):
    """
    Computes the mixture components for a given customer across all k values.
    MODEL: base measure H ~ Gamma(ha, hb), F(x|phi) ~ Poisson(phi)
    All components are calculated exactly in log-space and then exponentiated.
    
    returns: (Kmax,) vector; if new=True, returns a scalar
    """
    
    x = x.flatten()  # reshape to 1D, since gibbs routine passes in a 2D array
    
    # Calculate the case where k has no members
    fknew_cust = np.exp( -logg(x[i] + 1) + logg(x[i] + ha) - logg(ha) -
                         (x[i] + ha)*np.log(1 + hb) + ha*np.log(hb) )
    if new == True: return fknew_cust        
    
    x_kks = [x[k == kk] for kk in range(Kmax)]  # subset of customers eating kk
    xi_in = np.zeros(Kmax)                      # offset if x[i] is in this subset
    xi_in[k[i]] = 1
      
    # Compute (a,b) params from gamma kernel tricks done in fk function
    av = np.array(list(map(np.sum, x_kks))) - xi_in*x[i] + ha
    bv = np.array(list(map(len, x_kks))) - xi_in + hb
    fk_cust = np.exp( -logg(x[i] + 1) + logg(x[i] + av) - logg(av) -
                      (x[i] + av)*np.log(1 + bv) + av*np.log(bv) )
     
    return fk_cust


def pois_fk_tabl(jj, tt, x, j, t, k, Kmax, ha, hb, new=False):
    """
    Computes the mixture components for a given table across all k values.
    MODEL: base measure H ~ Gamma(ha, hb), F(x|phi) ~ Poisson(phi)
    All components are calculated exactly in log-space and then exponentiated.
    
    returns: (Kmax,) vector; if new=True, returns a scalar
    """
    
    x = x.flatten()  # reshape to 1D, since gibbs routine passes in a 2D array
    x_jt = x[np.logical_and(j == jj, t == tt)]
    kk = k[np.logical_and(j == jj, t == tt)]
    
    fknew_tabl = np.exp( -np.sum(logg(x_jt + 1)) + logg(np.sum(x_jt) + ha) - logg(ha) -
                         (np.sum(x_jt) + ha)*np.log(len(x_jt) + hb) + ha*np.log(hb) )
    # If table jt doesn't exist, just return the "new" mixture component
    if len(x_jt) == 0:
        print(f"WARNING: table {(jj, tt)} does not exist currently")
        new = True
    if new == True: return np.full(Kmax, fknew_tabl)
    
    x_kks = [x[k == kk] for kk in range(Kmax)]  # subset of customers at tables serving kk
    xjt_in = np.zeros(Kmax)                      # offset if table x_jt is in this subset
    xjt_in[kk[0]] = 1
      
    # Compute (a,b) params from gamma kernel tricks done in fk function
    av = np.array(list(map(np.sum, x_kks))) - xjt_in*np.sum(x_jt) + ha
    bv = np.array(list(map(len, x_kks))) - xjt_in*len(x_jt) + hb
    fk_tabl = np.exp( -np.sum(logg(x_jt + 1)) + logg(np.sum(x_jt) + av) - logg(av) -
                       (np.sum(x_jt) + av)*np.log(len(x_jt) + bv) + ha*np.log(bv) )
     
    return fk_tabl

In [202]:
def mnom_fk_cust(i, x, k, Kmax, L, ha, new=False):
    """
    Computes the mixture components for a given customer across all k values.
    MODEL: base measure H ~ Dirichlet(L, ha_1,...,ha_L),
                        F(x|phi) ~ Multinomial(n_ji, phi_1,...,phi_L)
    All components are calculated exactly in log-space and then exponentiated.
    
    returns: (Kmax,) vector; if new=True, returns a scalar
    """
    
    xi, ni = x[i, :], np.sum(x[i, :])
    log_con = logg(ni + 1) - np.sum(logg(xi + 1)) # term constant for all k
    # Calculate the case where k has no members
    fknew_cust = np.exp( log_con + np.sum(logg(xi + ha)) - logg(np.sum(xi + ha)) + 
                         logg(np.sum(ha)) - np.sum(logg(ha)) )
    if new == True: return fknew_cust        
    
    # Get subset of customers eating kk; each entry is a (#, L) matrix
    x_kks = [x[k == kk, :] for kk in range(Kmax)]  
    
    # Compute params from Dirichlet kernel tricks done in fk function
    a_bot = np.vstack([np.sum(x_kk, axis=0) for x_kk in x_kks]) + ha[None, :]    # (Kmax, L)
    a_bot[k[i], :] -= xi                         # offset if xi is in this subset
    a_top = a_bot + xi[None, :]
    fk_cust = np.exp( log_con + np.sum(logg(a_top), axis=1) - logg(np.sum(a_top, axis=1)) +
                      logg(np.sum(a_bot, axis=1)) - np.sum(logg(a_bot), axis=1) )
     
    return fk_cust


def mnom_fk_tabl(jj, tt, x, j, t, k, Kmax, L, ha, new=False):
    """
    Computes the mixture components for a given customer across all k values.
    MODEL: base measure H ~ Dirichlet(L, ha_1,...,ha_L),
                        F(x|phi) ~ Multinomial(n_ji, phi_1,...,phi_L)
    All components are calculated exactly in log-space and then exponentiated.
    
    returns: (Kmax,) vector; if new=True, returns a scalar
    """
    
    x_jt = x[np.logical_and(j == jj, t == tt), :]                                # (|T|, L)
    kk = k[np.logical_and(j == jj, t == tt)]
    n_jt = np.sum(x_jt, axis=1)                                                  # (|T|,)
    sum_jt = np.sum(x_jt, axis=0)                                                # (L,)
    log_con = np.sum(logg(n_jt + 1)) - np.sum(logg(x_jt + 1))    # term constant for all k
    
    fknew_tabl = np.exp( log_con + np.sum(logg(sum_jt + ha)) - logg(np.sum(sum_jt + ha)) + 
                         logg(np.sum(ha)) - np.sum(logg(ha)) )
    # If table jt doesn't exist, just return the "new" mixture component
    if x_jt.shape[0] == 0:
        print(f"WARNING: table {(jj, tt)} does not exist currently")
        new = True
    if new == True: return fknew_tabl       
    
    # Get subset of customers eating kk; each entry is a (#, L) matrix
    x_kks = [x[k == kk, :] for kk in range(Kmax)]
      
    # Compute params from Dirichlet kernel tricks done in fk function
    a_bot = np.vstack([np.sum(x_kk, axis=0) for x_kk in x_kks]) + ha[None, :]    # (Kmax, L)
    a_bot[kk[0], :] -= sum_jt                       # offset if table x_jt is in this subset
    a_top = a_bot + sum_jt[None, :]
    fk_tabl = np.exp( log_con + np.sum(logg(a_top), axis=1) - logg(np.sum(a_top, axis=1)) +
                      logg(np.sum(a_bot, axis=1)) - np.sum(logg(a_bot), axis=1) )

    return fk_tabl

In [6]:
class HDP:
    """
    Model implementing the Chinese Restaurant Franchise Process formulation of the HDP.
    
    CONSTRUCTOR PARAMETERS
    - gamma, alpha0: scaling parameters > 0 for base measures H and G0
    - f: string representing distribution of data; h is chosen to be conjugate
    - hypers: tuple of hyperparameter values specific to f/h scheme chosen
    
    PRIVATE ATTRIBUTES (volatile)
    - tk_map_: (J x Tmax) matrix of k values for each (j,t) pair
    - n_: (J x Tmax) matrix specifying counts of customers
    - m_: (J x Kmax) matrix specifying counts of tables
    - fk_cust_, fk_tabl_: functions to compute mixing components for Gibbs sampling
    
    PUBLIC ATTRIBUTES
    post_samples: (S x 3) matrix of (j, t, k) values for each data point i;
                  exists only after gibbs() has been called
    """
    
    def __init__(self, gamma=1, alpha0=1, f='multinomial', hypers=None):
        self.g_ = gamma
        self.a0_ = alpha0
        self.set_priors(f, hypers)
        
    def set_priors(self, f, hypers):
        """
        Initializes the type of base measure h_ and data-generation function f_.
        Also sets hypers_, the relevelant hyperparameters and
                  fk_routine_, the function to compute mixing components.
        """
        if f == 'poisson':
            # Specify parameters of H ~ Gamma(a,b)
            if hypers is None:
                self.hypers_ = (1,1)
            else: self.hypers_ = hypers
            self.fk_cust_ = pois_fk_cust
            self.fk_tabl_ = pois_fk_tabl
        
        if f == 'multinomial':
            if hypers is None:
                L = 2
                self.hypers_ = (L, np.full(L, 1/L))
            else: self.hypers_ = hypers
            self.fk_cust_ = mnom_fk_cust
            self.fk_tabl_ = mnom_fk_tabl
    
    
    def tally_up(self, it, which=None):
        """
        Helper function for computing maps and counts in gibbs().
        Given a current iteration in the post_samples attribute, does a full
        recount of customer/table allocations, updating n_ and m_.
        Set which = 'n' or 'm' to only tally up that portion
        """
        
        jt_pairs = self.post_samples[it,:,0:2]
        
        if which != 'm':
            # Count customers at each table (jt)
            cust_counts = pd.Series(map(tuple, jt_pairs)).value_counts()
            j_idx, t_idx = tuple(map(np.array, zip(*cust_counts.index)))
            self.n_ *= 0
            self.n_[j_idx, t_idx] = cust_counts
            
        if which != 'n':
            # First filter by unique tables (jt), then count tables with each k value
            jt_unique, k_idx = np.unique(jt_pairs, axis=0, return_index=True)
            jk_pairs = np.c_[self.post_samples[it, k_idx, 0],
                             self.post_samples[it, k_idx, 2]]
            #print(jk_pairs)
            tabl_counts = pd.Series(map(tuple, jk_pairs)).value_counts()
            #print(tabl_counts)
            j_idx, k_idx = tuple(map(np.array, zip(*tabl_counts.index)))
            self.m_ *= 0
            self.m_[j_idx, k_idx] = tabl_counts
    
    
    def draw_t(self, it, x, j, Tmax, Kmax, verbose):
        """
        Helper function which does the draws from the t_ij full conditional.
        Updates the counts and the samples matrices at iteration `it`.
        """
        
        t_next, k_next = self.post_samples[it,:,1], self.post_samples[it,:,2]
        # Cycle through each t value of each customer, conditioning on everything
        # Randomize the order in which updates occur
        for i in np.random.permutation(N):
            jj, tt0, kk0 = j[i], t_next[i], k_next[i]

            # Get vector of customer f_k values (dependent on model specification)
            old_mixes = self.fk_cust_(i, x, k_next, Kmax, *self.hypers_) 
            new_mixes = self.fk_cust_(i, x, k_next, Kmax, *self.hypers_, new=True) 
            # Calculate pointwise likelihoods p(x_ji | ...)
            M = np.sum(self.m_)
            Mk = np.sum(self.m_, axis=0)   # number of tables serving k
            lik = old_mixes @ (Mk / (M + self.g_)) + new_mixes * (self.g_ / (M + self.g_))

            cust_offset = np.zeros(Tmax)
            cust_offset[tt0] = 1
            old_t = (self.n_[jj, :] - cust_offset) * old_mixes[self.tk_map_[jj, :]]      
            new_t = self.a0_ * lik
            # If a table is in use, prob comes from old_t; otherwise, from new_t
            t_used = self.n_[jj, :] > 0
            t_dist = old_t * t_used.astype('int') + new_t * np.logical_not(t_used).astype('int')
            """TEMPORARY FIX (bug should be found later):
               Remove nans and add epsilon so that distribution is all positive"""
            t_dist[np.isnan(t_dist)] = 0
            t_dist += 1e-6

            tt1 = np.random.choice(Tmax, p=t_dist/np.sum(t_dist))
            t_next[i] = tt1
            self.tally_up(it, which='n')

            # If this table was previously unoccupied, we need to select a k
            if self.n_[jj, tt1] == 1 and tt0 != tt1:
                old_k = np.sum(self.m_, axis=0) * old_mixes
                new_k = self.g_ * new_mixes
                k_used = np.sum(self.m_, axis=0) > 0
                k_dist = old_k * k_used.astype('int') + new_k * np.logical_not(k_used).astype('int')
                """TEMPORARY FIX (bug should be found later):
                   Remove nans and add epsilon so that distribution is all positive"""
                k_dist[np.isnan(k_dist)] = 0
                k_dist += 1e-6

                kk1 = np.random.choice(Kmax, p=k_dist/np.sum(k_dist))
                self.tk_map_[jj, tt1] = kk1
                k_next[i] = self.tk_map_[jj, tt1]
            self.tally_up(it, which='m')

            if verbose: print(f"~ customer (j,i) = {(jj,i)}" +
                              f" moves table: {tt0} -> {t_next[i]}, k: {kk0} -> {k_next[i]}")
    
    
    def draw_k(self, it, x, j, Kmax, verbose):
        """
        Helper function which does the draws from the t_ij full conditional.
        Updates the counts and the samples matrices at iteration `it`.
        """
        
        t_next, k_next = self.post_samples[it,:,1], self.post_samples[it,:,2]
        # Similarly, cycle through the k values of each table
        j_idx, t_idx = np.where(self.n_ > 0)   # find the occupied tables
        for i in np.random.permutation(len(j_idx)):
            jj, tt = j_idx[i], t_idx[i]
            kk0 = self.tk_map_[jj, tt]

            # Get vector of table f_k values (dependent on model specification)
            old_mixes = self.fk_tabl_(jj, tt, x, j, t_next, k_next, Kmax, *self.hypers_) 
            new_mixes = self.fk_tabl_(jj, tt, x, j, t_next, k_next, Kmax, *self.hypers_, new=True) 

            tabl_offset = np.zeros(Kmax)
            tabl_offset[kk0] = 1
            old_k = (np.sum(self.m_, axis=0) - tabl_offset) * old_mixes
            new_k = self.g_ * new_mixes
            k_used = np.sum(self.m_, axis=0) > 0
            k_dist = old_k * k_used.astype('int') + new_k * np.logical_not(k_used).astype('int')
            """TEMPORARY FIX (bug should be found later):
               Remove nans and add epsilon so that distribution is all positive"""
            k_dist[np.isnan(k_dist)] = 0
            k_dist += 1e-6

            #print(f"{old_k.round(3)}\n{new_k.round(3)}\n{k_used}\n{k_dist.round(3)}")
            kk1 = np.random.choice(Kmax, p=k_dist/np.sum(k_dist))
            self.tk_map_[jj, tt] = kk1
            k_next[np.logical_and(j == jj, t_next == tt)] = kk1
            self.tally_up(it, which='m')

            if verbose: print(f"~~ table (j,t) = {(jj,tt)} changes dish: {kk0} -> {kk1}")
 
        
    def gibbs(self, x, j, iters, Tmax=5, Kmax=10, verbose=False):
        """
        Runs the Gibbs sampler to generate posterior estimates of t and k.
        x: data matrix, stored row-wise if multidimensional
        j: vector of group labels; must have same #rows as x
        iters: number of iterations to run
        Tmax: maximum number of clusters for each group
        Kmax: maximum number of atoms to draw from base measure H
        
        returns: this CFRP object with post_samples attribute
        """
            
        group_counts = pd.Series(j).value_counts()
        # number of tables cannot exceed size of max group
        J, N = np.max(j) + 1, len(j)
        self.n_ = np.zeros((J, Tmax))
        self.m_ = np.zeros((J, Kmax))
        self.post_samples = np.zeros((iters+1, N, 3), dtype='int')
        self.post_samples[:,:,0] = j
        
        # Set random initial values for t and k assignments
        t0, k0 = self.post_samples[0,:,1], self.post_samples[0,:,2]
        t0[:] = np.random.randint(1, Tmax, size=N)
        self.tk_map_ = np.random.randint(1, Kmax//2, (J, Tmax))
        self.tally_up(it=0, which='n')
        for jj in range(J):
            for tt in np.where(self.n_[jj, :] > 0)[0]:
                #print(f"mapping: {(jj, tt)} -> {self.tk_map_[jj, tt]}")
                k0[np.logical_and(j == jj, t0 == tt)] = self.tk_map_[jj, tt]
        self.tally_up(it=0, which='m')
        
        for s in range(iters):
            if verbose: print(f"----------------\n ITERATION {s+1}\n----------------")
            t_prev, k_prev = self.post_samples[s,:,1], self.post_samples[s,:,2]
            t_next, k_next = self.post_samples[s+1,:,1], self.post_samples[s+1,:,2]
            # Copy over the previous iteration as a starting point
            t_next[:], k_next[:] = t_prev, k_prev
            
            self.draw_t(s+1, x, j, Tmax, Kmax, verbose)
            self.draw_k(s+1, x, j, Kmax, verbose)
        
        return self  

In [7]:
# Simulated data (Poisson example)
N = 25
np.random.seed(0)
j = np.random.randint(0, 9, N)
x = np.random.poisson(j, N)
data = np.c_[x, j]

c = HDP(f='poisson', hypers=(1,10)).gibbs(x[:,None], j, iters=20)

In [14]:
# Simulated data (Multinomial example)
N, L, Kmax = 200, 100, 10
np.random.seed(1)
X = np.random.randint(0, 10, (N, L))
j = np.random.randint(0, 10, N)

c = HDP(f='multinomial', hypers=(L, np.full(L, 1/L))).gibbs(X, j, iters=10)

### Latent Topic Modeling Application

Below is an application of the above sampler using a multinomial data model.  The data is `final_project_data.csv`, produced by the modified preprocessing code in this directory, which contains a `(J, L)` matrix in which entry `(j,l)` contains the count of word `l` in document `j`, with the corresponding words given in the column names.

For the Dirichlet prior here, we use the observed distribution of the corpus vocabulary over all documents.  Customers could be encoded in four different ways to compare performance:
+ As a single word (such that `f` is categorical)
+ As a set of all identical words within a given document (each row of the data matrix has one entry, but the value can vary)
+ As a set of all words in a single sentence
+ As the entire document (essentially making this a non-hierarchical DP)

Since this algorithm has not been optimized, only a subset of the full dataframe is used for now.

In [177]:
import pandas as pd

full_df = pd.read_csv('final_project_data.csv', index_col=0, dtype='int')
vocab = full_df.columns

In [186]:
def expand_doc(doc_in):
    """Expands a row (passed in a series) into a dataframe in which each
       row contains only the counts of one unique word."""
    
    doc_in = doc_in.iloc[0,:]
    words_used = doc1[doc1 > 0]
    doc_out = pd.DataFrame(np.zeros((len(words_used), len(vocab)), dtype='int'),
                           columns=vocab, dtype='int')
    for i, word in enumerate(words_used.index):
        doc_out.loc[i, word] = words_used[word]
    return doc_out

J_ID = 'document#'
if J_ID not in full_df.columns:
    full_df.insert(0, column=J_ID, value=full_df.index)

Jmax = 10
wordset_df = full_df.iloc[:Jmax,:].groupby(J_ID).apply(expand_doc)

In [198]:
ji_indices = wordset_df.index.to_frame()
j = np.array(ji_indices[J_ID])
X = np.array(wordset_df)
# Get the corresponding word each ji is associated with
ji_words = vocab[np.where(X > 0)[1]]

Tmax, Kmax = np.max(ji_indices[1]), 20
# Get a prior distribution over the vocabulary from selected documents
L, h_alpha = X.shape[1], np.sum(X, axis=0)
h_alpha = alpha / np.sum(alpha)
iters = 10

In [199]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    %time hdp = HDP(f='multinomial', hypers=(L, h_alpha)).gibbs(X, j, 100, Tmax, Kmax)

Wall time: 8min 5s


In [216]:
# Given seating assignment in final iteration, cluster customers with same k value
k_final = hdp.post_samples[-1,:,2]
clusters = pd.DataFrame({'doc': j, 'word': ji_words, 'cluster': k_final})
for k in set(k_final):
    clusters_k = clusters[clusters['cluster'] == k]
    print(f"-----------\nk = {k} (size {clusters_k.shape[0]})")
    print(clusters_k)

-----------
k = 0 (size 14)
     doc         word  cluster
50     1      sterile        0
52     1       detect        0
56     1  fertilizing        0
58     1    mutagenic        0
78     2         self        0
111    3      isolate        0
150    5      elegans        0
152    5     nematode        0
171    5      isolate        0
174    5  application        0
277    9  temperature        0
282    9       lethal        0
290    9      sterile        0
299    9     applying        0
-----------
k = 1 (size 10)
     doc            word  cluster
62     2        nematode        1
67     2     temperature        1
93     3       mutations        1
94     3         mutants        1
101    3      conditions        1
117    3         optimal        1
165    5       treatment        1
215    7        specific        1
271    9  caenorhabditis        1
298    9       mutagenic        1
-----------
k = 2 (size 23)
     doc         word  cluster
13     0   techniques        2
21     0      i

## *( 5.3 )* Posterior Sampling by Direct Assignment